In [1]:
#Imports
from scipy.stats import qmc
import math
import numpy
import wordle, worker
from multiprocessing import Pool
import os
import json

In [2]:
#Seed the generator so that results are consistent
sobolGen = qmc.Sobol(2, scramble=True, seed=0)
haltonGen = qmc.Halton(2, scramble=True, seed=0)
numpy.random.seed(0)

In [3]:
#Set up
pairsToGen = 2048

wordle_small = open("data/wordle_small.txt")
answerList = [word.strip() for word in wordle_small]
wordle_small.close()

wordle_large = open("data/wordle_large.txt")
possibleList = [word.strip() for word in wordle_large] + answerList
wordle_large.close()

l_bounds = [0,0]
u_bounds = [len(answerList) for _ in range(0,2)]

In [4]:
sobolSample = sobolGen.random(pairsToGen)
sobolSampleScaled = qmc.scale(sobolSample, l_bounds, u_bounds)
sobolWords = [(answerList[math.floor(x)], answerList[math.floor(y)])for x,y in sobolSampleScaled]

In [5]:
haltonSample = haltonGen.random(n=pairsToGen)
haltonSampleScaled = qmc.scale(haltonSample, l_bounds, u_bounds)
haltonWords = [(answerList[math.floor(x)], answerList[math.floor(y)])for x,y in haltonSampleScaled]

In [6]:
randomSample = numpy.random.randint(0, len(answerList), size=(pairsToGen,2))
randomWords = [(answerList[x], answerList[y]) for x,y in randomSample]

In [7]:
sobolCount = {}
for tup in sobolWords:
    for word in tup:
        if word in sobolCount.keys():
            sobolCount[word] += 1
        else:
            sobolCount[word] = 1
print("Unique sobol words: " + str(len(sobolCount)))

Unique sobol words: 2085


In [8]:
haltonCount = {}
for tup in haltonWords:
    for word in tup:
        if word in haltonCount.keys():
            haltonCount[word] += 1
        else:
            haltonCount[word] = 1
print("Unique halton words: " + str(len(haltonCount)))

Unique halton words: 2281


In [9]:
randomCount = {}
for tup in randomWords:
    for word in tup:
        if word in randomCount.keys():
            randomCount[word] += 1
        else:
            randomCount[word] = 1
print("Unique random words: " + str(len(haltonCount)))

Unique random words: 2281


In [10]:
response = wordle.score("chess", "swiss")
remaining = wordle.filter_words(answerList, "swiss", response)
len(remaining)

19

In [ ]:
evsMap = {}
with Pool(processes=15) as p:
    print("Starting processing...")
    evs = p.starmap(worker.evCalc, [(i, haltonWords, answerList) for i in possibleList])
    print("Processing done!")
    for i in range(len(evs)):
        evsMap[possibleList[i]] = evs[i]
print(evsMap)

Starting processing...


In [ ]:
vals = list(evsMap.values())
vals.sort()
temp = vals[0:10]
res = [key for key in evsMap if evsMap[key] in temp]
res

In [ ]:
with open("analysis/wordle_all_halton.json", "w") as outfile:
    json.dump(evsMap, outfile)

In [ ]:
evsMapSobol = {}
with Pool(processes=15) as p:
    print("Starting processing...")
    evs = p.starmap(worker.evCalc, [(i, sobolWords, answerList) for i in possibleList])
    print("Processing done!")
    for i in range(len(evs)):
        evsMapSobol[possibleList[i]] = evs[i]
print(evsMapSobol)

In [ ]:
vals = list(evsMapSobol.values())
vals.sort()
temp = vals[0:10]
res = [key for key in evsMapSobol if evsMapSobol[key] in temp]
res

In [ ]:
with open("analysis/wordle_all_sobol.json", "w") as outfile:
    json.dump(evsMapSobol, outfile)

In [ ]:
evsMapRandom = {}
with Pool(processes=15) as p:
    print("Starting processing...")
    evs = p.starmap(worker.evCalc, [(i, randomWords, answerList) for i in possibleList])
    print("Processing done!")
    for i in range(len(evs)):
        evsMapRandom[possibleList[i]] = evs[i]
print(evsMapRandom)

In [ ]:
vals = list(evsMapRandom.values())
vals.sort()
temp = vals[0:10]
res = [key for key in evsMapRandom if evsMapRandom[key] in temp]
res

In [ ]:
with open("analysis/wordle_all_random.json", "w") as outfile:
    json.dump(evsMapRandom, outfile)